In [1]:
import numpy as np
import pandas as pd
import sklearn.datasets
import lightgbm
import multiprocessing
import time
from numba import jit, cuda
import numba
from tqdm import tqdm

In [2]:
data=sklearn.datasets.load_boston()
df=pd.DataFrame(data['data'],columns=data['feature_names'])
df_work=pd.concat([df,pd.DataFrame(data['target'],columns=['price'])],axis=1)
df_work.shape


(506, 14)

In [3]:
class Boosted_Feature_extractor:

  def __init__(self,n_feature,model,train_data,train_label,val_data,val_label,estimator=100):
    self.n_feature=n_feature
    self.model=model
    self.train_data=train_data
    self.train_label=train_label
    self.val_data=val_data
    self.val_label=val_label
    self.estimator=estimator
    self.pool_score={}
    self.grid_score={}
    self.grid=None
    self.pred=None

  def performScoring(scoring_actual_field, scoring_predicted_field, scoring_arguments):
        actuals = scoring_actual_field.tolist()
        predicted = scoring_predicted_field.tolist()
        test_data_len=len(scoring_actual_field)
        merror = mean_squared_error(actuals, predicted)
        RMSE = (merror**.5)
        days =scoring_arguments
        pred_rnd = np.round_(predicted)
        report = "MSE :" + ' ' + str(np.around(merror,decimals=2)) + "\n" + "\n" + "RMSE :" + ' ' + str(np.around(RMSE,decimals=2)) + " days" + "\n" + "\n"
        # Calculating accuracy for days
        for day in days:
            count = 0
            for j in zip(actuals, pred_rnd):
                if ((abs(j[0] - j[1]) <= int(day))):
                    count += 1
            accuracy= (count / test_data_len * 100)
            append_string = "+/-" + str(day) + ' ' + 'Days' + " :" + ' ' + str(np.around(accuracy,decimals=2)) + "%"  "\n" + "\n"
            report = report + append_string
        return np.around(accuracy,decimals=2)
  
  def selection_grid(self):
     self.grid=pd.concat([pd.DataFrame(np.random.randint(0,2,size=(len(self.n_feature),self.estimator))) for i in range(4)]).reset_index(drop=True)

  #sub_grid :tuple (index, sub_df)
  def model_training(self,sub_grid):
    print('inside trainer')
    selection=sub_grid[1].loc[sub_grid[0]:sub_grid[0]+len(self.n_feature)-1]
    ind=selection[selection==1].index
    if ind[0]>=len(self.n_feature):
      ind-=sub_grid[0]
    print(ind)
    self.model.fit(self.train_data.iloc[:,ind],self.train_label)
    print(1)
    self.pred=self.model.predict(self.val_data.iloc[:,ind])
    scr=self.performScoring(self.val_label,self.pred,[3])
    self.pool_score[scr]=self.train_label.iloc[:,ind].columns
    print(2)
    
  def perform_selection(self):
    # selfselection_grid(self.n_feature,self.estimator)
    for i in tqdm(self.grid.columns):
        sub_grid=[self.grid[i]]*4
        pool_index=list(range(0,self.grid.shape[0],len(self.n_feature)))
        inputs=zip(pool_index,sub_grid)
        pool = multiprocessing.Pool()
        print('start')
        pool.map(self.model_training, inputs)
        print('stop')
        pool.close()
        pool.join()
    self.grid_score[max(self.pool_score.keys())]=self.pool_score[max(self.pool_score.keys())]
    print(self.grid_score)
    self.pool_score={}
    return (max(self.grid_score.keys()),self.grid_score[max(self.grid_score.keys())])  

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_work.iloc[:,:-1],df_work['price'],test_size=.2,random_state=42)

In [5]:
total_features=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']
model=lightgbm.LGBMRegressor()

In [6]:
bfe=Boosted_Feature_extractor(total_features,model,x_train,y_train,x_test,y_test)


In [7]:
bfe.selection_grid()


In [ ]:
bfe.perform_selection()

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

start


In [ ]:
import multiprocessing
import time
  
# square function
def square(x):
    return x * x
  
  
if __name__ == '__main__':
  
    # multiprocessing pool object
    pool = multiprocessing.Pool()
  
    # pool object with number of element
    pool = multiprocessing.Pool(processes=4)
  
    # input list
    inputs = [0, 1, 2, 3, 4]
  
    # map the function to the list and pass
    # function and input list as arguments
    outputs = pool.map(square, inputs)
  
    # Print input list
    print("Input: {}".format(inputs))
  
    # Print output list
    print("Output: {}".format(outputs))